# Auto-Graded Checkpoints Demo

This notebook demonstrates the new auto-grading functionality that provides automatic scoring with weighted rules.

**Note:** If you encounter `AttributeError` about missing methods like `add_scoring_rule`, please restart the Jupyter kernel (Kernel → Restart) to ensure the latest version of the module is loaded.

In [ ]:
# Import and reload to ensure we have the latest version
import importlib
import sys

# Remove any cached modules
modules_to_remove = [key for key in sys.modules.keys() if key.startswith('jupyter_lab_progress')]
for module in modules_to_remove:
    del sys.modules[module]

# Now import fresh
from jupyter_lab_progress import LabProgress, LabValidator, show_info
import pandas as pd
import numpy as np

show_info("Modules reloaded to ensure latest version with auto-grading is used!")

## Setting Up Auto-Graded Lab

Let's create a lab that will be automatically graded based on various criteria.

In [ ]:
# Create progress tracker and validator
progress = LabProgress(
    steps=[
        "Import Libraries",
        "Load Data",
        "Data Preprocessing",
        "Feature Engineering",
        "Model Training"
    ],
    lab_name="Machine Learning Lab"
)

validator = LabValidator(progress_tracker=progress)

# Check if auto-grading functionality is available
show_info("Checking auto-grading functionality...")
validator.check_auto_grading_available()

## Adding Scoring Rules

Define weighted scoring rules that will automatically grade the lab work.

In [ ]:
# Add scoring rule for library imports
validator.add_scoring_rule(
    name="Libraries Imported",
    weight=0.1,  # 10% of total grade
    checker={
        'type': 'variable_exists',
        'var_name': 'pd',
        'expected_type': type(pd)
    },
    description="Check if pandas is imported as pd"
)

# Add scoring rule for data loading
def check_data_loaded(globals_dict):
    """Custom checker function that returns (success, score)"""
    if 'df' not in globals_dict:
        return False, 0
    
    df = globals_dict['df']
    if not isinstance(df, pd.DataFrame):
        return False, 0
    
    # Partial credit based on data size
    if len(df) >= 1000:
        return True, 100
    elif len(df) >= 500:
        return True, 80
    elif len(df) >= 100:
        return True, 60
    else:
        return True, 40

validator.add_scoring_rule(
    name="Data Loaded",
    weight=0.2,  # 20% of total grade
    checker=check_data_loaded,
    description="Check if DataFrame 'df' is loaded with sufficient data",
    partial_credit=True
)

# Add scoring rule for preprocessing
def check_preprocessing(globals_dict):
    """Check if data is properly preprocessed"""
    if 'df' not in globals_dict:
        return False, 0
    
    df = globals_dict['df']
    score = 0
    
    # Check for no missing values (50 points)
    if df.isnull().sum().sum() == 0:
        score += 50
    
    # Check if numeric columns exist (50 points)
    if len(df.select_dtypes(include=[np.number]).columns) > 0:
        score += 50
    
    return score > 0, score

validator.add_scoring_rule(
    name="Data Preprocessing",
    weight=0.25,  # 25% of total grade
    checker=check_preprocessing,
    description="Check data cleaning and preprocessing steps"
)

# Add scoring rule for feature engineering
validator.add_scoring_rule(
    name="Feature Engineering",
    weight=0.25,  # 25% of total grade
    checker={
        'type': 'variable_exists',
        'var_name': 'features',
        'expected_type': (list, np.ndarray, pd.DataFrame)
    },
    description="Check if features variable is created"
)

# Add scoring rule for model
def check_model(globals_dict):
    """Check if a model is trained and has expected methods"""
    if 'model' not in globals_dict:
        return False, 0
    
    model = globals_dict['model']
    score = 0
    
    # Check for fit method (50 points)
    if hasattr(model, 'fit'):
        score += 50
    
    # Check for predict method (50 points)
    if hasattr(model, 'predict'):
        score += 50
    
    return score > 0, score

validator.add_scoring_rule(
    name="Model Training",
    weight=0.2,  # 20% of total grade
    checker=check_model,
    description="Check if a machine learning model is properly instantiated"
)

## Student Work Simulation

Let's simulate a student completing the lab tasks.

In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

progress.mark_done("Import Libraries")
show_info("Libraries imported successfully!")

In [ ]:
# Step 2: Load data (creating sample data for demo)
np.random.seed(42)
df = pd.DataFrame({
    'feature1': np.random.randn(1500),
    'feature2': np.random.randn(1500),
    'feature3': np.random.choice(['A', 'B', 'C'], 1500),
    'target': np.random.choice([0, 1], 1500)
})

# Add some missing values to demonstrate preprocessing
df.loc[df.sample(50).index, 'feature1'] = np.nan

progress.mark_done("Load Data")
show_info(f"Data loaded: {len(df)} rows, {len(df.columns)} columns")

In [ ]:
# Run auto-grading at this point (partial completion)
show_info("Running auto-grading on current progress...")
results = validator.run_auto_grading(globals())

In [ ]:
# Step 3: Data preprocessing
# Fill missing values
df['feature1'].fillna(df['feature1'].mean(), inplace=True)

# Convert categorical to numeric
df = pd.get_dummies(df, columns=['feature3'])

progress.mark_done("Data Preprocessing")
show_info("Data preprocessing completed!")

In [ ]:
# Step 4: Feature engineering
features = df.drop('target', axis=1)
target = df['target']

# Create additional engineered features
features['feature1_squared'] = features['feature1'] ** 2
features['feature_interaction'] = features['feature1'] * features['feature2']

progress.mark_done("Feature Engineering")
show_info(f"Features created: {features.shape[1]} features")

In [ ]:
# Step 5: Model training
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

progress.mark_done("Model Training")
show_info("Model trained successfully!")

## Final Auto-Grading

Run the complete auto-grading to see the final score.

In [ ]:
# Run final auto-grading
final_results = validator.run_auto_grading(globals())

## Export Grading Report

In [ ]:
# Generate text report
report = validator.export_grading_report()
print("\nText Report:\n")
print(report)

In [ ]:
# View validation history
history_df = validator.get_validation_history_df()
show_info(f"Total validations performed: {len(history_df)}")
history_df.tail()

## Custom Scoring Functions

You can create complex scoring functions that check multiple criteria.

In [ ]:
# Example of a comprehensive scoring function
def comprehensive_model_check(globals_dict):
    """Advanced model validation with multiple criteria"""
    score = 0
    feedback = []
    
    # Check if model exists (20 points)
    if 'model' not in globals_dict:
        return False, 0
    
    model = globals_dict['model']
    score += 20
    
    # Check if it's a sklearn model (20 points)
    if hasattr(model, 'get_params'):
        score += 20
        feedback.append("✓ Valid sklearn model")
    
    # Check if model is fitted (30 points)
    if hasattr(model, 'n_features_in_'):
        score += 30
        feedback.append("✓ Model is fitted")
    
    # Check accuracy if test data exists (30 points)
    if all(var in globals_dict for var in ['X_test', 'y_test']):
        try:
            accuracy = model.score(globals_dict['X_test'], globals_dict['y_test'])
            if accuracy > 0.6:
                score += 30
                feedback.append(f"✓ Good accuracy: {accuracy:.2%}")
            else:
                score += 15
                feedback.append(f"⚠ Low accuracy: {accuracy:.2%}")
        except:
            feedback.append("✗ Could not evaluate model accuracy")
    
    # Display feedback
    for item in feedback:
        print(item)
    
    return score > 50, score

# Add this comprehensive rule
validator.add_scoring_rule(
    name="Comprehensive Model Check",
    weight=0.0,  # Set to 0 to not affect current grading
    checker=comprehensive_model_check,
    description="Advanced model validation with multiple criteria"
)

# Test the comprehensive check
success, score = validator.validate_with_score("Comprehensive Model Check", globals_dict=globals())
print(f"\nComprehensive check score: {score}/100")

## Summary

The auto-grading system provides:
- **Weighted scoring rules** for flexible grading schemes
- **Partial credit** support for nuanced evaluation
- **Custom validation functions** for complex checks
- **Visual feedback** with color-coded results
- **Grade calculation** with letter grades
- **Export capabilities** for record keeping
- **History tracking** for audit trails